In [1]:
from SMT_model import *
from time import time as time_clock
from z3 import IntNumRef
import json

In [2]:
#Demo with instance 01
Path = ('../Instances/inst10.dat')

timeout = 300


In [3]:
#Read the file from the path
with open(Path, 'r') as file:
    data = file.read()

instance = data_parsing(data)
print(instance)

{'m': 10, 'n': 13, 'l': [185, 190, 200, 180, 200, 190, 200, 180, 195, 190], 's': [22, 17, 10, 8, 14, 12, 17, 19, 25, 25, 6, 21, 6], 'D': [[0, 21, 86, 14, 84, 72, 24, 54, 83, 70, 8, 91, 42, 57], [21, 0, 71, 35, 70, 51, 16, 75, 62, 91, 29, 70, 57, 52], [86, 71, 0, 100, 39, 70, 87, 137, 81, 73, 78, 103, 128, 33], [14, 35, 100, 0, 98, 86, 38, 49, 97, 56, 22, 105, 29, 71], [84, 70, 39, 98, 0, 109, 86, 135, 120, 57, 76, 129, 126, 27], [63, 51, 70, 77, 109, 0, 49, 117, 11, 133, 71, 64, 90, 103], [24, 16, 87, 38, 86, 49, 0, 78, 60, 94, 32, 67, 41, 60], [63, 84, 137, 49, 135, 135, 87, 0, 146, 79, 59, 154, 65, 108], [74, 62, 81, 88, 120, 11, 60, 128, 0, 144, 82, 68, 94, 114], [70, 91, 73, 56, 57, 142, 94, 79, 153, 0, 63, 161, 72, 40], [8, 29, 78, 22, 76, 80, 32, 59, 91, 63, 0, 99, 50, 49], [91, 70, 133, 105, 129, 64, 67, 145, 68, 161, 99, 0, 91, 122], [42, 57, 128, 29, 126, 90, 41, 65, 94, 72, 50, 91, 0, 99], [57, 52, 33, 71, 27, 103, 60, 108, 114, 40, 49, 122, 99, 0]]}


In [4]:
def run_smt(instance, timeout, sb=False):
    generation_start_time = time_clock()

    # Build the SMT model
    o, x, max_distance = stm_model(instance, timeout, sb)
    generation_duration = time_clock() - generation_start_time
    o.set("timeout", int(timeout - generation_duration) * 1000)

    # Minimize the objective
    obj = o.minimize(max_distance)
    res = o.check()  # Check satisfiability
    final_time = int(time_clock() - generation_start_time)

    if res == sat:
        try:
            # Format the solution if satisfiable
            result_formatted = format_solution(instance, o.model(), x)
            return result_formatted, True, obj.value(), final_time
        except Exception as e:
            return str(e), False, obj.value()
    elif res == unknown:
        try:
            model = o.model()
            if model:  # Check if a model exists
                # Format the partial solution
                result_formatted = format_solution(instance, model, x)
                best_objective = model.eval(max_distance, model_completion=True)
                return (
                    result_formatted,
                    False,
                    best_objective.as_long() if best_objective.is_int() else float(best_objective.as_decimal(5)),
                    final_time
                )
            else:
                # No model available, return fallback message
                return "No solution found", False, None
        except Exception as e:
            return f"unknown\nError retrieving model: {e}", False, None
    elif res == unsat:
        return "unsat", False, None
    else:
        return "unknown", False, None

In [10]:
print(run_smt(instance, timeout, False))

sat
[x_7_6 = 13,
 x_2_3 = 13,
 x_5_5 = 13,
 x_0_4 = 13,
 x_2_4 = 13,
 x_6_11 = 13,
 x_8_1 = 13,
 x_9_2 = 13,
 x_3_6 = 13,
 x_3_2 = 13,
 x_9_1 = 0,
 x_5_2 = 13,
 distances = Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(Store(K(Int,
                                        Store(Store(Store(Store(Store(Store(Store(...,
                                        ...,
                                        ...),
                                        12,
                                        65),
                                        2,
                                        137),
                                        13,
                                        108),
                                        9,
                                        79),
                                        3,
                                        49),
                                        10,
                                        59)),
                              

In [5]:
def all_solutions(solvers, solutions):
    def convert(obj):
        if isinstance(obj, IntNumRef):
            return obj.as_long()
        if isinstance(obj, list):
            return [convert(i) for i in obj]
        return obj
    
    output_data = []
    sol_index=0
    for solver in solvers:
        solution = solutions[sol_index]
        if solution == "unsat":
            print("Problem is unsatisfiable for solver", solver)
        elif solution == "unknown":
            print("Problem is unknown for solver", solver)
        elif solution[0] == "No solution found":
            print("No solution found for solver", solver)
        else:
            output_data.append({
                solver:
                    {
                    "time": convert(solution[3]),
                    "optimal": convert(solution[1]),
                    "obj": convert(solution[2]),
                    "sol": convert(solution[0])
                    }
                })
        sol_index += 1
        
    return output_data

In [6]:
def save_solution(solvers ,solutions, path):
    output_data = all_solutions(solvers, solutions)
    with open(path, 'w') as file:
        if output_data != []:
            print(output_data)
            json.dump(output_data, file, indent=4)

In [7]:
#Create a function that runs the SMT model for all the instances and saves the solutions in json files.
def run_all_instances():
    for i in range(1, 21):
        if i < 10:
            Path = f'../Instances/inst0{i}.dat'
        else:
            Path = f'../Instances/inst{i}.dat'
        if i < 30:
            with open(Path, 'r') as file:
                data = file.read()
            instance = data_parsing(data)
            solution_smt = run_smt(instance, timeout, False)
            solution_smt_sb = run_smt(instance, timeout, True)
            save_solution(["smt", "smt_sb"], [solution_smt, solution_smt_sb], f'Solutions/instance{i}.json')

In [ ]:
run_all_instances()

[{'smt': {'time': 1, 'optimal': True, 'obj': 14, 'sol': [[4, 3, 1], [2, 5, 6]]}}, {'smt_sb': {'time': 0, 'optimal': True, 'obj': 14, 'sol': [[4, 3, 1], [2, 5, 6]]}}]
[{'smt': {'time': 16, 'optimal': True, 'obj': 226, 'sol': [[1, 4], [7], [2], [6, 8], [5], [3, 9]]}}, {'smt_sb': {'time': 8, 'optimal': True, 'obj': 226, 'sol': [[9], [2], [8, 6], [7], [4, 1], [5, 3]]}}]
[{'smt': {'time': 2, 'optimal': True, 'obj': 12, 'sol': [[5, 6, 3], [2, 4], [1, 7]]}}, {'smt_sb': {'time': 0, 'optimal': True, 'obj': 12, 'sol': [[5, 4, 2], [7, 3], [1, 6]]}}]
[{'smt': {'time': 10, 'optimal': True, 'obj': 220, 'sol': [[10], [8], [3, 1], [7], [9, 5], [6], [4], [2]]}}, {'smt_sb': {'time': 14, 'optimal': True, 'obj': 220, 'sol': [[9], [6], [4], [3], [1, 5], [2], [8], [10, 7]]}}]
[{'smt': {'time': 0, 'optimal': True, 'obj': 206, 'sol': [[2], [1, 3]]}}, {'smt_sb': {'time': 0, 'optimal': True, 'obj': 206, 'sol': [[2], [1, 3]]}}]
[{'smt': {'time': 1, 'optimal': True, 'obj': 322, 'sol': [[8], [6], [7, 5, 4], [3], [